In [3]:
# imports
!pip install datasets tokenizers
import os
import math
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from pathlib import Path
from datasets import load_dataset
from tqdm import tqdm
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


# Download Dataset

In [5]:
# English/Malay pairs from HuggingFace
train_dataset = load_dataset("Helsinki-NLP/opus-100", "en-ms", split='train')
validation_dataset = load_dataset("Helsinki-NLP/opus-100", "en-ms", split='validation')

# Limit the amount of data for training purposes
raw_train_dataset, rt_to_skip = random_split(train_dataset, [1500, len(train_dataset) - 1500])
raw_validation_dataset, vt_to_skip = random_split(validation_dataset, [50, len(validation_dataset) - 50])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/132k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/57.1M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/132k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

# Tokenizer

In [6]:
!mkdir -p /content/tokenizer_en
!mkdir -p /content/tokenizer_my

In [7]:
# Returns a generator list from a dataset of the given language
def get_ds_iterator(raw_train_dataset, lang):
  for data in raw_train_dataset:
    yield data["translation"][lang]

# Create English source tokenizer
tokenizer_en = Tokenizer(BPE(unk_token="[UNK]"))
trainer_en = BpeTrainer(min_frequency=2, special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])
# Pre-tokenizer to split input into words
tokenizer_en.pre_tokenizer = Whitespace()
tokenizer_en.train_from_iterator(get_ds_iterator(raw_train_dataset, "en"), trainer=trainer_en)
tokenizer_en.save("./tokenizer_en/tokenizer_en.json")

# Create Malay source tokenizer
tokenizer_my = Tokenizer(BPE(unk_token="[UNK]"))
trainer_my = BpeTrainer(min_frequency=2, special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])
# Pre-tokenizer to split input into words
tokenizer_my.pre_tokenizer = Whitespace()
tokenizer_my.train_from_iterator(get_ds_iterator(raw_train_dataset, "ms"), trainer=trainer_my)
tokenizer_my.save("./tokenizer_my/tokenizer_my.json")

In [8]:
# Retrieve tokenizers we made
tokenizer_en = Tokenizer.from_file("./tokenizer_en/tokenizer_en.json")
tokenizer_my = Tokenizer.from_file("./tokenizer_my/tokenizer_my.json")

# Get the vocab sizes
source_vocab_size = tokenizer_en.get_vocab_size()
target_vocab_size = tokenizer_my.get_vocab_size()

In [9]:
max_seq_len_source = 0
max_seq_len_target = 0

# Calculate the max sequence length in the training dataset for source/target
for data in raw_train_dataset:
  enc_ids = tokenizer_en.encode(data["translation"]["en"]).ids
  dec_ids = tokenizer_my.encode(data["translation"]["ms"]).ids
  max_seq_len_source = max(max_seq_len_source, len(enc_ids))
  max_seq_len_target = max(max_seq_len_target, len(dec_ids))

print("Source vocab max sequence length:", max_seq_len_source)
print("Target vocab max sequence length:", max_seq_len_target)

Source vocab max sequence length: 134
Target vocab max sequence length: 99


In [10]:
# Standard max sequence length for training, with buffer for padding, the classification token, unknown tokens, separator tokens, etc.
max_seq_len = 155

# Dataset and Dataloader

In [19]:
# Causal mask to hide future tokens
def causal_mask(size):
  # Square matrix with ones in the lower triangle: size x size
  mask = torch.triu(torch.ones(1, size, size), diagonal=1).type(torch.int)
  return mask == 0

In [26]:
# Encode raw dataset to be processed by the model
class EncodeDataset(Dataset):
  def __init__(self, raw_dataset, max_seq_len):
    super().__init__()
    self.raw_dataset = raw_dataset
    self.max_seq_len = max_seq_len

  def __len__(self):
    return len(self.raw_dataset)

  def __getitem__(self, index):
    # Fetch data (in both English and Malay) for the given index
    raw_text = self.raw_dataset[index]

    # Separate text into source and target
    source_text = raw_text["translation"]["en"]
    target_text = raw_text["translation"]["ms"]

    # Encode text
    source_text_encoded = tokenizer_en.encode(source_text).ids
    target_text_encoded = tokenizer_my.encode(target_text).ids

    # Convert CLS, SEP, and PAD to their vocab index id using the tokenizer
    # Start of sentence token
    CLS_ID = torch.tensor([tokenizer_my.token_to_id("[CLS]")], dtype=torch.int64)
    # End of sentence token
    SEP_ID = torch.tensor([tokenizer_my.token_to_id("[SEP]")], dtype=torch.int64)
    # Padding token
    PAD_ID = torch.tensor([tokenizer_my.token_to_id("[PAD]")], dtype=torch.int64)

    # Amount to pad the encoded text
    num_source_padding = self.max_seq_len - len(source_text_encoded) - 2
    num_target_padding = self.max_seq_len - len(target_text_encoded) - 1
    encoder_padding = torch.tensor([PAD_ID] * num_source_padding, dtype=torch.int64)
    decoder_padding = torch.tensor([PAD_ID] * num_target_padding, dtype=torch.int64)

    # CLS + source encoding + SEP + padding
    encoder_input = torch.cat([CLS_ID, torch.tensor(source_text_encoded, dtype=torch.int64), SEP_ID, encoder_padding], dim=0)
    # CLS + target encoding + padding
    decoder_input = torch.cat([CLS_ID, torch.tensor(target_text_encoded, dtype=torch.int64), decoder_padding], dim=0)

    # target encoding + SEP + padding
    target_label = torch.cat([torch.tensor(target_text_encoded, dtype=torch.int64), SEP_ID, decoder_padding], dim=0)

    # Masks to ignore padding
    encoder_mask = (encoder_input != PAD_ID).unsqueeze(0).unsqueeze(0).int()
    # Apply causal mask to decoder mask, so that the decoder can't see future tokens when predicting the next token
    decoder_mask = (decoder_input != PAD_ID).unsqueeze(0).unsqueeze(0).int() & causal_mask(decoder_input.size(0))

    return {
        "encoder_input": encoder_input,
        "decoder_input": decoder_input,
        "target_label": target_label,
        "encoder_mask": encoder_mask,
        "decoder_mask": decoder_mask,
        "source_text": source_text,
        "target_text": target_text
    }

# Create encoded datasets
train_ds = EncodeDataset(raw_train_dataset, max_seq_len)
val_ds = EncodeDataset(raw_validation_dataset, max_seq_len)

# Create dataloaders to use in the model
train_dataloader = DataLoader(train_ds, batch_size=5, shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)

In [28]:
'''
encoder_input: Encoded source text with start and end of sentence tokens and padding
decoder_input: Encoded target text with start of sentence token and padding
target_label: Encoded target text with padding
encoder_mask: Mask to ignore padding in the encoder input
decoder_mask: (Causal) mask to ignore padding in the decoder input
source_text: Original source text
target_text: Original target text
'''
train_ds.__getitem__(0)

{'encoder_input': tensor([   2,    6,   33,  101, 1700, 1553,   67,  467,   16,    6,    3,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           

# Input Embedding and Positional Encoding

In [29]:
class EmbeddingLayer(nn.Module):
  def __init__(self, d_model, vocab_size):
    super().__init__()
    self.d_model = d_model
    # Embedding layer to map token ids to embeddings (vocab_size x d_model)
    self.embedding = nn.Embedding(vocab_size, d_model)

  def forward(self, input):
    # Multiply embedding by the sqrt(d_model) to normalize the output
    embedding_output = self.embedding(input) * math.sqrt(self.d_model)
    return embedding_output

In [47]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_seq_len, dropout_rate):
    super().__init__()
    self.d_model = d_model
    self.dropout = nn.Dropout(dropout_rate)

    # Init positional encodings, positions
    pe = torch.zeros(max_seq_len, d_model)
    pos = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
    # 1 / (10000 ** (2 * i / d_model))
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000) / d_model))

    # Apply div term to positional encodings, with sin/cos depending on even/odd dimensions
    pe[:, 0::2] = torch.sin(pos * div_term)
    pe[:, 1::2] = torch.cos(pos * div_term)

    # Add batch dimension
    # pe: 1 x seq_len x d_model
    pe = pe.unsqueeze(0)
    # Ensure that the positional encodings are a part of the model, but not trainable
    self.register_buffer("pe", pe)

  def forward(self, input_embedding):
    # input_embedding: batch_size x seq_len x d_model
    input_embedding = input_embedding + (self.pe[:, :input_embedding.shape[1], :]).requires_grad_(False)
    return self.dropout(input_embedding)